In [4]:
import numpy as np
import random as ran
import matplotlib.pyplot as plt
from random import SystemRandom, randint
from miller_rabin import is_prime

In [12]:
def decompose(n):
    """Generates a dictionary representing the prime decomposition."""
    factors={}
    current_number=n                            #divide current_number by the factors found until it reaches 1
    while current_number > 1:
        p=smallest_factor(current_number)
        if p in factors.keys():                 #if p is not a new factor, increase the power
            factors[p]+=1
        else:
            factors[p]=1                        #if p is a new factor, create a new entry
        current_number = current_number//p
    return factors

def smallest_factor(n):
    """Returns the smallest factor of a positive integer n."""
    sqrt=n**0.5
    i=2
    while i<=sqrt:
        if n%i==0:
            return i                            #If we get here, return i as the value.
        i+=1
    return n   

def random_prime(bit_length):
    '''
    Returns a cryptographically secure random numbber 
    of bit_length many (binary) bits 
    '''
    while True:
        p = SystemRandom().getrandbits(bit_length)  
        if is_prime(p):
            return p


def random_prime1(n_bit, t, trails = 500):
    '''Returns a uniform n-bit prime, with probability at most 2^(-t) getting a composite,
    the higher the trails the less chance of failing to generate a prime but takes longer'''
    for i in range(0, trails):
        p = np.random.randint(0,2,n_bit-2)
        b = "".join(str(i) for i in p)
        my_int = int("1" + b + "1", 2) #Want at least n bit and gives odd        
        if is_prime_miller_rabin(my_int, t):
            return my_int
    return "Fail"

def is_prime_miller_rabin(N,t = 40):
    '''If N is prime, test always outputs True(prime)
    If N is composite outputs False(not prime) except with probability at most 2^(-t)'''
    
    if N == 2:
        return True # Prime
    
    if N == 3:
        return True # Prime
    
    if N % 2 == 0:
        return False # Comp
    
    if N < 2:
        return False # Comp
    
    d, s = calc_d_s(N)
    for j in range(0,t):
        a = ran.randrange(2,N-1)
        if is_composite(N, d, s, a):
            return False #Comp
    return True #Prime

def calc_d_s(N):
    s = 0
    Num = N - 1
    while Num%2 == 0:
        Num //= 2
        s += 1
    d = Num
    return int(d), int(s)

def is_composite(N, d, s, a):
    '''Evaulate a^d != +-1 mod(N), and, a^(d*2^r) != -1 mod(N) for r in {1,2,...,s-1},
    However, if output False does not mean prime, it could be a strong liar'''
    remain = pow(a, d, N)
    if (remain != 1) and (remain != N - 1):
        for r in range(1,s):
            remain = pow(remain, 2, N)
            if remain != N-1:
                pass
            else:
                return False    
    else:
        return False
    
    return True

In [19]:
from time import process_time   


def generate_N(k,n):
    alist = ([])
    for i in range(2*n):
        alist.append(random_prime(k,40))
    x = np.array_split(alist,n)
    blist = ([])
    for j in range(n):
        blist.append(np.product(x[j]))
    return blist

numbers = generate_N(25,5)


sample_size = len(numbers)

start = process_time()           
for n in numbers:
    decompose(n)
end = process_time()            

total_time = end - start
average_time = total_time / sample_size

print("\nThe time taken was {:.2f} seconds".format(total_time))
print("The average time per number was", end = " ") 
print("{:.2f} seconds".format(average_time))  



The time taken was 197.19 seconds
The average time per number was 39.44 seconds


In [18]:
numbers = generate_N(25,5)
sample_size = len(numbers)

start = process_time()           
for n in numbers:
    decompose(n)
end = process_time()            

total_time = end - start
average_time = total_time / sample_size

print("\nThe time taken was {:.2f} seconds".format(total_time))
print("The average time per number was", end = " ") 
print("{:.2f} seconds".format(average_time)) 


The time taken was 119.62 seconds
The average time per number was 23.92 seconds
